In [1]:
import numpy as np
from matplotlib import pyplot as plt


# width      = 1000 * 540 * 1.5 / 2
# spacesteps = 1000

# x = np.linspace(0, width, spacesteps)
# dx = width / spacesteps

In [2]:
from math import floor
from tqdm import tqdm

def trapezium(func, bounds, delta, display=False):
    result = delta * (func(bounds[0]) + func(bounds[1])) / 2

    steps = (bounds[1] - bounds[0]) / delta

    if display:
        for i in tqdm(range(1, floor(steps))):
            result += delta * func(bounds[0] + (delta * i))

    else:
        for i in range(1, floor(steps)):
            result += delta * func(bounds[0] + (delta * i))

    return result
    
def simpson(func, bounds, delta):
    result = delta * (func(bounds[0]) + func(bounds[1])) / 3

    steps = (bounds[1] - bounds[0]) / delta
    for i in range(1, floor(steps)):
        add = 2 * delta * func(bounds[0] + (delta * i))

        if i % 2 != 0:
            add *= 2

        result += add / 3

    return result

In [3]:
trap = trapezium(lambda x: x ** 2, (0, 10), 0.01)
simp = simpson  (lambda x: x ** 2, (0, 10), 0.01)
print(f'{trap} vs {simp}')

333.33349999999996 vs 333.33333333333337


In [4]:
from math import sqrt, pi, exp, cos, sin

S = 3.6e-5
L = 10000
N = 0.01
rho_s = 1
D = 50000
D_t = 10000
h = 100000

mode = 1

# modal variables
A_j = sqrt(2 / (rho_s * (N ** 2) * D))

# wavespeed (squared)
c2 = ((N * D) ** 2) / (((mode * pi) ** 2) + ((D ** 2)/(4 * (h ** 2)))) # wavespeed

# calculating S_j = S0 * sigma
if ((mode * D_t / D) - 1) == 0:
    S_j = A_j * (rho_s / 2) * D_t
else:
    S_A = np.sin((np.pi) * ((mode * D_t / D) - 1)) / ((mode * D_t / D) - 1)
    S_B = np.sin((np.pi) * ((mode * D_t / D) + 1)) / ((mode * D_t / D) + 1)
    S_j = A_j * (rho_s / 2) * (D_t / np.pi) * (S_A - S_B)

S_j = S_j * S


def F(x):
    return 1 / np.cosh(x / L) ** 2

In [5]:
xMax = 100 * L
dx = xMax / 1000

#xMax = (1000 * 540 * 1.5 / 2)
#dx = xMax / 1000
    
def FourierTransform(func, k):
    # e^ix   = cos(x) + i*sin(x)
    # e^-ikx = e^i(-kx) = cos(-kx) + i*sin(-kx) = cos(kx) - i*sin(kx)

    real = lambda x: cos(k*x)  * func(x)
    im   = lambda x: -sin(k*x) * func(x)

    return (trapezium(real, (-xMax, xMax), dx), trapezium(im, (-xMax, xMax), dx))




def FourierR(func, k):
    real = lambda x: cos(k*x)  * func(x)

    return trapezium(real, (-xMax, xMax), dx)

def FourierI(func, k):
    im   = lambda x: -sin(k*x) * func(x)

    return trapezium(im, (-xMax, xMax), dx)

def InverseFourierTransform(func, x):
    # t = 1 if x < 1 else x
    kMax = 30 / L
    dk = kMax / 1000#kMax / 1000

    # e^ix   = cos(x) + i*sin(x)
    # e^ikx = e^i(kx) = cos(kx) + i*sin(kx)

    # func = a + ib
    # acos, a i sin, b i cos, b ii sin -> real: acos - bsin, im: asin + bcos

    (a, b) = func

    real = lambda k: (a(k) * cos(k*x)) - (b(k) * sin(k*x))
    #im   = lambda k: (a(k) * sin(k*x)) + (b(k) * cos(k*x))

    resultR = trapezium(real, (-kMax, kMax), dk, True) / (2 * pi)
    resultI = 0#trapezium(im, (-kMax, kMax), dk, True) / (2 * pi)

    return (resultR, resultI)

In [6]:
for x in [t * 1000 for t in [50, 100, 200, 300, 405]]:
    true = F(x)

    fR = lambda k: FourierR(F, k)
    fI = lambda k: FourierI(F, k)

    fourier = InverseFourierTransform((fR, fI), x)

    print(f'x={x}: {true} | {fourier[0]} | {str(round(100 * (true - fourier[0]) / true, 2))}%')

100%|██████████| 1999/1999 [00:13<00:00, 145.16it/s]


x=50000: 0.00018158323094380667 | 0.0001815832309438202 | -0.0%


100%|██████████| 1999/1999 [00:13<00:00, 145.62it/s]


x=100000: 8.244614455767395e-09 | 8.24461460469295e-09 | -0.0%


100%|██████████| 1999/1999 [00:13<00:00, 144.77it/s]


x=200000: 1.6993417021166355e-17 | 6.486360867172979e-16 | -3716.98%


100%|██████████| 1999/1999 [00:13<00:00, 144.66it/s]


x=300000: 3.502604305078608e-26 | 1.0167078329917665e-15 | -2902719646300.22%


100%|██████████| 1999/1999 [00:14<00:00, 142.37it/s]

x=405000: 2.6558708798322934e-35 | -2.1481261694951011e-16 | 8.088217638166001e+20%


In [7]:
a = 5
example      = lambda x: exp(-a * abs(x))
exampleTilde = lambda k: (2 * a) / ((a ** 2) + k ** 2)

testTildeR   = lambda k: FourierR(example, k)
testTildeI   = lambda k: FourierI(example, k)


for k in [0, 1, 5, 10, 50, 100]:
    print(f'Fourier values: (k={k})')
    true    = exampleTilde(k)
    fourier = testTildeR(k) # we know im should be 0

    print(f'{true} | {fourier} | {str(round(100 * (true - fourier) / true, 2))}%')

    print(f'Inverted values: (x={k})')
    true     = example(k)
    inverted = InverseFourierTransform((testTildeR, testTildeI), k)

    print(f'{true} | {inverted} | {str(round(100 * (true - inverted[0]) / true, 2))}%')

    print(f'------------------------------')


Fourier values: (k=0)
0.4 | 1000.0 | -249900.0%
Inverted values: (x=0)


100%|██████████| 1999/1999 [00:03<00:00, 626.47it/s]


1.0 | (0.9549296585513906, 0) | 4.51%
------------------------------
Fourier values: (k=1)
0.38461538461538464 | 1000.0 | -259900.0%
Inverted values: (x=1)


 19%|█▉        | 383/1999 [00:00<00:02, 619.37it/s]


KeyboardInterrupt: 